## Package and data imports

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#sklearn
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score


In [130]:
#Data import
df = pd.read_csv('./datasets/train_cleaned.csv')

# Helper functions

## Categorical Variable Encoding

Several categorical variables in this data set may convey information but may add too many parameters to our model (and lead to overfitting) if one-hot encoded.  Instead of one-hot encoding all of these categorical variables, we'll want to be able to encode them in a simpler form.  The function below carries out one possible such encoding; it turns a categorical variable into a numeric variable that only takes on the values 1, 0 and -1 based on whether they're associated with higher or lower values of our target variable `SalePrice`.  1 corresponds to "above average", 0 to "about average", and -1 to "below average."  See the example below, after the function's code.

In [127]:
def above_below_mid(df, col, target):
    '''
    Inputs:
    -A Pandas dataframe df
    -The name of a (categorical) column in df
    -The name of the target (numeric) column to be predicted by modeling
    
    Output:
    Returns a dicticonary that sorts each value appearing in that column
    to 1 if it's associated with above-median target value, -1 if it's
    associated with a below-median target value, and 0 if it's associated
    with an approximately-median target value.
    
    Specifically, this function does:
    1. For each value 's' that is taken on in that column,
     find the conditional median of the target value given that
     df[col]=='s';
    2. Find the range of these conditional medians, i.e.
     [LOWEST CND MEDIAN, HIGHEST CND MEDIAN];
    3. For each value 's' that is taken on in that column,
      -Appends {'s':1} to the output dict if the conditional median
      for 's' was in the top third of this range;
      -Appends {'s':0} if it was in the middle thrid;
      -Appends {'s':-1} if it was in the bottom third.
    '''
    
    output={}
    meds={}
    pop_med = df[target].median()
    
    #Make a dictionary of conditional medians for each value in the column
    for name in df[col].unique():
        
        #Find conditional median of target for that value
        cnd_med = df[ df[col]==name ][target].median()
        meds[name] = cnd_med
            
    #Find the most extreme conditional medians
    highest_med = max(meds.values())
    lowest_med = min(meds.values())
    diff = highest_med - lowest_med

    #Group the column's values based on whether their conditional target mean
    #is in the lower thrid (-1), upper third (1), or middle third (0)
    
    for name in meds.keys():
        if meds[name] < lowest_med + diff/3:
            output[name] = -1
        elif meds[name] > highest_med - diff/3:
            output[name] = 1
        else:
            output[name] = 0
    
    return output
    

In [129]:
#An example
above_below_mid(df, col='Condition 1', target='SalePrice')

{'RRAe': -1,
 'Norm': 0,
 'PosA': 1,
 'Artery': -1,
 'Feedr': -1,
 'PosN': 1,
 'RRAn': 0,
 'RRNe': -1,
 'RRNn': 1}

Those strings that are assigned -1 can be thought of as associated with below-average home prices; those assigned +1 are associated with above-average home prices; and those assigned 0 are associated with approximately average prices.

Looking at the [data description](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt) for the feature `Condition 1`, we can see that these assignments make a good deal of sense.  The value 'RRAe' is mapped to -1, which fits with the idea that being *adjacent to a railrod* probably leads to lower home prices.  The value 'Norm' is mapped to 0, which means that "Normal" (according to the data description) houses have approximately average prices.  'PosA' is described as "Adjacent to postive off-site feature," and we see that it's mapped to 1, corresponding to higher-than-average home prices.

### Regression fitting and evaluation

In [67]:
def regression(df, features, target,
        test_size=.25, random_state=None, folds=5, outputs=['crossval']):
    '''
    Inputs:
    df: A Pandas dataframe
    features: A list of column names in df that you want included as regressors
    target: The name of a column in df that you want as the dependent variable
    
    Parameters:
    test_size: Proportion of df to use as a test data set. Defaults to .25.
    random_state: Random state for train-test split. Defaults to None.
    folds: The number of folds to be used for cross-val scoring of the model.
    outputs: A list of strings indicating the items this function should output.
    
        You can enter the following arguments (defaults to only 'crossval'):
        
        'crossval': The cross-val scores of the model on the UNION of
                    train+test sets
        'r2_test': R-Squared value from the test data set
        'r2_train': R-squared value from the training data set
        'rmse_test': Root mean-squared error of model on test data
        'rmse_train': Root mean-squared error of model on train data
        
        'lr': The sklearn linear model fitted on the UNION of
              training and test sets
        'coefs': A Pandas series of the fitted coefs from the UNION of
                 train+test sets
        'intercept': The fitted intercept coef from the UNION of train+test sets
        
        'lr_train': The sklearn linear model fitted on the training set
        'coefs_train': A Pandas series of the fitted coefs from the training set
        'intercept_train': The fitted intercept coef from the training set
        
        'train_indices': An array of the indices of observations used in train set
        'test_indices': An array of the indices of observations used in test set
    
    Output:
    Splits df into training and test sets.  Trains a linear model with
    X=df[features] and y=df[target], then returns the desired items listed
    in 'outputs'.  
    '''
    
    X = df[features]
    y = df[target]
    
    #Initialize a dictionary for outputs
    returns = {}
    
    
    
    
    
    
    #If the only desired output is crossval, then we don't need to train a model ourselves
    if outputs == ['crossval']:
        return {'crossval' : cross_val_score(LinearRegression(), X, y, cv=folds)}
    
    
    
    
    
    
    
    
    
    
    #If none of the desired outputs include 'train' or 'test', then we only need
    #to fit a model on the ENTIRE data set (no train/test splitting needed)
    
    notrain = 'train' not in ''.join(outputs)
    notest = 'test' not in ''.join(outputs)
    
    if notrain and notest:
    
        #Fit a linear model on the entire data set
        lr = LinearRegression()
        lr.fit(X, y)
        
        #For each type of desired output, add it to our output dictionary
        for x in outputs:
            if x=='lr':
                returns[x] = lr
            if x=='coefs':
                returns[x] = lr.coef_
            if x=='intercept':
                returns[x] = lr.intercept_
            if x=='crossval':
                returns[x] = cross_val_score(LinearRegression(), X, y, cv=folds)
        return returns
    
    
    
    
    
    
    #Now check if ALL the desired outputs (other than 'crossval') reference 'train'
    #or 'test'.  If so, then we don't need to fit a regression on the WHOLE
    #data set - only on the train set.
    
    #Initialize an indicator to see if all desired outputs (except possibly 'crossval')
    #contain 'train' or 'test'
    indicator = 1
    
    for x in outputs:
        if x=='crossval':
            continue
        elif 'train' in x:
            continue
        elif 'test' in x:
            continue
        else:
            indicator = 0
            break
    
    #If no desired outputs require a whole-data-set fitting...
    if indicator == 1:
        
        #Split the data set
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                            random_state=random_state,
                                                            test_size=test_size)
        #Fit a linear model on the training set
        lr_train = LinearRegression()
        lr_train.fit(X_train, y_train)
    
        #For each type of desired output, add it to our output dictionary
        for x in outputs:
            if x=='crossval':
                returns[x] = cross_val_score(LinearRegression(), X, y, cv=folds)
            if x=='r2_test':
                returns[x] = lr_train.score(X_test, y_test)
            if x=='r2_train':
                returns[x] = lr_train.score(X_train, y_train)
            if x=='rmse_test':
                returns[x] = metrics.mean_squared_error(y_test, lr_train.predict(X_test), squared=False)
            if x=='rmse_train':
                returns[x] = metrics.mean_squared_error(y_train, lr_train.predict(X_train), squared=False)
            if x=='lr_train':
                returns[x] = lr_train
            if x=='coefs_train':
                returns[x] = lr_train.coef_
            if x=='intercept_train':
                returns[x] = lr_train.intercept_
            if x=='train_indices':
                returns[x] = X_train.index
            if x=='test_indices':
                returns[x] = X_test.index
        return returns
    
    
    
    
    
    
    
    
    #If neither of the above conditions is met, then we need to fit regression models
    #on BOTH the whole data set and a test set.
    else:
        
        #Split the data set
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                            random_state=random_state,
                                                            test_size=test_size)
        #Fit a linear model on the training set
        lr_train = LinearRegression()
        lr_train.fit(X_train, y_train)
        
        #Fit a separate linear model on the WHOLE data set
        lr = LinearRegression()
        lr.fit(X, y)
        
        #For each type of desired output, add it to our output dictionary
        for x in outputs:
            if x=='crossval':
                returns[x] = cross_val_score(LinearRegression(), X, y, cv=folds)
            if x=='r2_test':
                returns[x] = lr_train.score(X_test, y_test)
            if x=='r2_train':
                returns[x] = lr_train.score(X_train, y_train)
            if x=='rmse_test':
                returns[x] = metrics.mean_squared_error(y_test, lr_train.predict(X_test), squared=False)
            if x=='rmse_train':
                returns[x] = metrics.mean_squared_error(y_train, lr_train.predict(X_train), squared=False)
            if x=='lr_train':
                returns[x] = lr_train
            if x=='coefs_train':
                returns[x] = lr_train.coef_
            if x=='intercept_train':
                returns[x] = lr_train.intercept_
            if x=='train_indices':
                returns[x] = X_train.index
            if x=='test_indices':
                returns[x] = X_test.index
            if x=='lr':
                returns[x] = lr
            if x=='coefs':
                returns[x] = lr.coef_
            if x=='intercept':
                returns[x] = lr.intercept_
        return returns
    

## First attempt: Some basic numeric features

We'll run a regression on those numeric features that at first glance seem most likely to contribute to sale price.  We will try to not include features that are likely to be muticollinear.

In [28]:
#Extract the numeric features
numerics = []

for feature in df.dtypes.index:
    if (df.dtypes[feature] == 'int64') or (df.dtypes[feature] == 'int64'):
        numerics.append(feature)
        
numerics

['Id',
 'PID',
 'MS SubClass',
 'Lot Area',
 'Overall Qual',
 'Overall Cond',
 'Year Built',
 'Year Remod/Add',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Full Bath',
 'Half Bath',
 'Bedroom AbvGr',
 'Kitchen AbvGr',
 'TotRms AbvGrd',
 'Fireplaces',
 'Wood Deck SF',
 'Open Porch SF',
 'Enclosed Porch',
 '3Ssn Porch',
 'Screen Porch',
 'Pool Area',
 'Misc Val',
 'Mo Sold',
 'Yr Sold',
 'SalePrice']

Let's remove the features that are numeric but don't capture any real information (such as 'Id' and 'PID'), as well as some features that capture complex aggregations of may facts (and are hence likely to be multicollinear with the other variables), such as `Ms SubClass`:

In [70]:
numerics.remove('Id')
numerics.remove(

['PID',
 'MS SubClass',
 'Lot Area',
 'Overall Qual',
 'Overall Cond',
 'Year Built',
 'Year Remod/Add',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Full Bath',
 'Half Bath',
 'Bedroom AbvGr',
 'Kitchen AbvGr',
 'TotRms AbvGrd',
 'Fireplaces',
 'Wood Deck SF',
 'Open Porch SF',
 'Enclosed Porch',
 '3Ssn Porch',
 'Screen Porch',
 'Pool Area',
 'Misc Val',
 'Mo Sold',
 'Yr Sold',
 'SalePrice']

Some of these features are likely to be multicollinear: e.g. combining living area + kitchens + baths is likely to yiled something close to total square footage.

In [69]:
#Make a list of features that are likely to not be multicollinear
features = ['Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add',
     '1st Flr SF', '2nd Flr SF', 'Fireplaces', 'Wood Deck SF', 'Open Porch SF',
     'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val']

X=df[features]
X.head()

,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,1st Flr SF,2nd Flr SF,Fireplaces,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val
0,13517,6,8,1976,2005,725,754,0,0,44,0,0,0,0,0
1,11492,7,5,1996,1997,913,1209,1,0,74,0,0,0,0,0
2,7922,5,7,1953,2007,1057,0,0,0,52,0,0,0,0,0
3,9802,5,5,2006,2007,744,700,0,100,0,0,0,0,0,0
4,14235,6,8,1900,1993,831,614,0,0,59,0,0,0,0,0


In [33]:
y = df['SalePrice']

In [60]:
#Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [43]:
#Fit the linear model
lr = LinearRegression()
lr.fit(X_train, y_train)

preds_train = lr.predict(X_train)
preds_test = lr.predict(X_test)

resids_train = y_train - preds_train
resids_test = y_test - preds_test

In [36]:
#Examine the coefficients
pd.Series(lr.coef_, index = X.columns)

Lot Area              1.691142
Overall Qual      20274.572133
Overall Cond       4163.902057
Year Built          462.386595
Year Remod/Add      256.882231
1st Flr SF           81.871243
2nd Flr SF           45.605186
Fireplaces         4352.571382
Wood Deck SF         36.319224
Open Porch SF        43.423401
Enclosed Porch       34.672620
3Ssn Porch           48.207552
Screen Porch         92.614159
Pool Area           -54.380518
Misc Val             -0.386996
dtype: float64

In [42]:
#Find R-squared on training data
lr.score(X_train, y_train)

0.8397014033882123

In [38]:
#Compare to R-squared on testing data
lr.score(X_test, y_test)

0.8064774414113615

In [39]:
#Cross validation
cross_val_score(lr, X_train, y_train).mean()

0.8322810537016668

In [41]:
#Cross-val score on entire dataset
cross_val_score(lr, X, y).mean()

0.825343231305113

In [44]:
#Compute RMSE on test set
metrics.mean_squared_error(y_test, preds_test, squared=False)

36669.135216026305

## Make a Kaggle submission

In [47]:
kaggle = pd.read_csv('./datasets/test_cleaned.csv')

In [48]:
X_to_pred = kaggle[features]
X_to_pred.head()

,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,1st Flr SF,2nd Flr SF,Fireplaces,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val
0,9142,6,8,1910,1950,908,1020,0,0,60,112,0,0,0,0
1,9662,5,4,1977,1977,1967,0,0,170,0,0,0,0,0,0
2,17104,7,5,2006,2006,664,832,1,100,24,0,0,0,0,0
3,8520,5,6,1923,2006,968,0,0,0,0,184,0,0,0,0
4,9500,6,5,1963,1963,1394,0,2,0,76,0,0,185,0,0


In [49]:
kaggle_preds = lr.predict(X_to_pred)

In [51]:
kaggle['SalePrice'] = kaggle_preds
output = kaggle[['Id', 'SalePrice']]
output.head()

,Id,SalePrice
0,2658,157497.463267
1,2718,199232.286392
2,2414,211507.277632
3,1989,106525.637038
4,625,189379.482082


In [52]:
output.to_csv('./datasets/sub_first_try.csv', index=False)

In [68]:
regression(df, features, 'SalePrice', folds=4, random_state=42, test_size=.2,
          outputs = ['crossval', 'r2_train', 'r2_test', 'rmse_train', 'rmse_test'])

{'crossval': array([0.80511485, 0.84598097, 0.83563872, 0.83034346]),
 'r2_train': 0.8423946437710884,
 'r2_test': 0.7849207582618265,
 'rmse_train': 31415.17720261409,
 'rmse_test': 37165.113652123786}